In [1]:
from hausdorf_dist import get_points, write_to_laz, hausdorff_distance, filter_distance
import numpy as np
from scipy.spatial import cKDTree
import pdal
from numpy.lib import recfunctions as rfn


READ_PIPELINE = """
{{
    "pipeline": [
        {{
            "type": "readers.ept",
            "filename": "{path}",
            "bounds": "{bounds}"
        }},
        {{
            "type":"filters.crop",
            "polygon":"{wkt}"
        }}
    ]
}}
"""

WRITE_PIPELINE = """
{{
    "pipeline": [
        {{
            "type": "writers.las",
            "filename": "{path}",
            "extra_dims": "all"
        }}
    ]
}}
"""

print( 'done')

done


In [2]:
path_a10_2018 = '/var/data/rws/data/2018/entwined/ept.json'
path_a10_2019 = '/var/data/rws/data/2019/amsterdam_entwined/ept.json'

In [3]:
xmin, ymin, xmax, ymax = (117964.8,483470.4, 118026.0,483508.9)
bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
# wkt = 'POLYGON((118086.081 483485.581,118074.903 483485.732,118065.581 483486.119,118056.542 483486.938,118047.62 483488.153,118039.427 483489.528,118030.678 483491.222,118019.016 483493.788,118008.571 483496.71,118001.12 483499.283,117992.838 483502.526,117984.157 483506.123,117976.422 483509.641,117967.756 483514.054,117960.784 483517.997,117954.183 483521.932,117947.879 483526.024,117947.11 483513.278,117951.731 483510.126,117960.923 483504.716,117967.435 483501.457,117975.959 483497.407,117984.403 483493.631,117990.872 483490.943,117997.782 483488.186,118006.963 483484.931,118015.923 483482.174,118015.708 483481.366,118002.074 483482.955,118000.135 483483.181,117983.637 483486.304,117970.765 483489.303,117967.445 483490.282,117955.523 483495.844,117947.998 483499.457,117943.606 483496.587,117942.175 483493.478,117941.047 483490.418,117940.646 483487.213,117945.653 483484.839,117950.911 483483.156,117966.037 483479.008,117981.592 483475.324,117997.092 483472.27,118014.693 483469.609,118031.886 483467.622,118044.005 483466.636,118057.819 483466.064,118067.851 483465.89,118080.102 483466.03,118085.655 483466.22,118084.219 483467.035,118084.421 483470.17,118084.868 483473.458,118084.257 483476.936,118081.318 483480.858,118082.203 483484.306,118086.081 483485.581))' 
# wkt = 'POLYGON((118004.2 483483.8,118004.2 483495.2,118023.3 483495.2,118023.3 483483.8,118004.2 483483.8))'
wkt = 'POLYGON((117964.8 483470.4,117964.8 483508.9,118026 483508.9,118026 483470.4,117964.8 483470.4))'  # larger square

In [4]:
point_cloud_1 = get_points(path_a10_2018, bounds, wkt)
points_1 = point_cloud_1[['X', 'Y', 'Z']]
points_1 = np.array(points_1[['X', 'Y', 'Z']].tolist())

In [5]:
point_cloud_2 = get_points(path_a10_2019, bounds, wkt)
points_2 = point_cloud_2[['X', 'Y', 'Z']]
points_2 = np.array(points_2[['X', 'Y', 'Z']].tolist())

In [ ]:
reference_array = np.array(point_cloud_1[['X', 'Y', 'Z']].tolist())
compared_array = np.array(point_cloud_2[['X', 'Y', 'Z']].tolist())

In [14]:
point_cloud_1[['X','Y','Z','Red','Green','Blue']]

array_1 = np.array(point_cloud_1[['X', 'Y', 'Z']].tolist())
array_2 = np.array(point_cloud_2[['X', 'Y', 'Z']].tolist())
haus_distances = hausdorff_distance(array_1, array_2)

out_array = rfn.append_fields(point_cloud_2[['X','Y','Z','Red','Green','Blue']], 'Distance', haus_distances)

out_array

masked_array(data=[(118011.998, 483498.33, 25.993, 29951, 38911, 45311, 0.10868762578617167),
                   (118012.054, 483498.176, 25.993, 23039, 27135, 30975, 0.0827345151515279),
                   (117965.187, 483494.207, 27.425, 30975, 38399, 46335, 0.20769448717811007),
                   ...,
                   (118022.812, 483482.961, 8.387, 36863, 30207, 26879, 0.03911521441827577),
                   (118022.81, 483482.86, 8.391, 35839, 29183, 25343, 0.020099751231932594),
                   (118022.818, 483482.433, 8.4, 35839, 30463, 26111, 0.01802775637530468)],
             mask=[(False, False, False, False, False, False, False),
                   (False, False, False, False, False, False, False),
                   (False, False, False, False, False, False, False), ...,
                   (False, False, False, False, False, False, False),
                   (False, False, False, False, False, False, False),
                   (False, False, False, False, False, Fal

In [7]:
out_points = filter_distance(point_cloud_1, point_cloud_2, 0.05)

/home/arnot/projects/rws_pc/hausdorf_dist.py:83: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_array = rfn.append_fields(compared_cloud[['X','Y','Z','Red','Green','Blue']], 'Distance', haus_distances)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [12]:
max_dist = 0.05 

haus_distances = hausdorff_distance(points_1, points_2)
out_array = rfn.append_fields(point_cloud_2[['X','Y','Z','Red','Green','Blue']], 'Distance', haus_distances)

filtered_out_array = out_array[out_array['Distance'] < max_dist]
write_to_laz(filtered_out_array, '/var/data/rws/data/output/test.laz')

In [ ]:


out_array = np.swapaxes(np.array(
                [point_cloud_2['X'],
               point_cloud_2['Y'],
               point_cloud_2['Z'],
               point_cloud_2['Red'],
               point_cloud_2['Green'],
               point_cloud_2['Blue'],
               haus_distances]
        ),0,1)

filtered_out_array = total[total[:,-1] < max_dist]
print(filtered_out_array.shape)
filtered_out_array.dtype = {'names':['X', 'Y', 'Z', 'Red', 'Green', 'Blue', 'Distances'], 
                            # 'formats':['<f8','<f8','<f8','<u8','<u8','<u8','<f8']}
                            'formats':[np.float64, np.float64 ,np.float64 ,np.uint64,np.uint64,np.uint64,np.float64]}

# point_cloud_1
# np.save('/var/data/rws/data/output/test.npy',filtered_out_array)

write_to_laz(filtered_out_array, '/var/data/rws/data/output/test.laz')

In [ ]:
print(point_cloud_2[['Red','Green','Blue']])
print(filtered_out_array[['Red','Green','Blue']]
     )

In [ ]:
for name in filtered_out_array.dtype.names:
    print(name)
    print(filtered_out_array[name].min())
    print(filtered_out_array[name].max())
    print(filtered_out_array[name].mean())

In [ ]:
tst = np.load('/var/data/rws/data/output/test.npy')
print(tst.shape)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
_ = plt.hist(total2[:,-1], bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

In [ ]:
READ_PIPELINE = """
{
    "pipeline": [
        {
            "type": "readers.las",
            "filename": "/var/data/rws/data/output/test.laz"
        }
    ]
}
"""
pipeline = pdal.Pipeline(READ_PIPELINE)
pipeline.validate()
pipeline.execute()
point_cloud = pipeline.arrays[0]

In [15]:
print(xmin,ymin,xmax,ymax)

117964.8 483470.4 118026.0 483508.9
